In [26]:
import os
import pandas as pd

In [27]:
def getData():
    if not os.path.exists('dataset.csv'):
        dfs = []
        files = os.listdir('datasets')
        for file in files:
            if file.endswith('.csv'):
                print(file)
                df = pd.read_csv(os.path.join('datasets', file))
                dfs.append(df)

        df = pd.concat(dfs, ignore_index=True)
        
        df.to_csv('dataset.csv', index=False)
        return df
    else:
        return pd.read_csv('dataset.csv')

In [28]:
data = getData()

C:\Users\tarik\AppData\Local\Temp\ipykernel_20208\1811304781.py:16: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv('dataset.csv')


In [29]:
data.head()

,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1.0,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1.0,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1.0,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0.0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1.0,1


In [30]:
data.rename(columns={'urls': 'url_count'}, inplace=True)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185474 entries, 0 to 185473
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   sender     184519 non-null  object 
 1   receiver   178821 non-null  object 
 2   date       182172 non-null  object 
 3   subject    182495 non-null  object 
 4   body       184840 non-null  object 
 5   label      184688 non-null  float64
 6   url_count  184688 non-null  object 
dtypes: float64(1), object(6)
memory usage: 9.9+ MB


In [32]:
data.describe()

,label
count,184688.000000
mean,0.485619
std,0.499794
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


## Preprocessing

In [33]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tarik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tarik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tarik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
data.dropna(subset=['label'], inplace=True)
data.dropna(subset=['body'], inplace=True)
data['date'].fillna(data['date'].mode()[0], inplace=True)
data['url_count'] = pd.to_numeric(data['url_count'], errors='coerce').fillna(0).astype(int)

text_columns = ['sender', 'receiver', 'subject']
data[text_columns] = data[text_columns].fillna('Unknown')

In [35]:
data.drop_duplicates(inplace=True)

In [36]:
data['email_type'] = data['subject'].apply(lambda x: 1 if re.search('^re:', x, re.IGNORECASE) else 2 if re.search('^fwd:', x, re.IGNORECASE) else 0)

data['subject'] = data['subject'].apply(lambda x: re.sub('^re:', '', x, flags=re.IGNORECASE))
data['subject'] = data['subject'].apply(lambda x: re.sub('^fwd:', '', x, flags=re.IGNORECASE))

In [37]:
# Function to clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Lowercasing and cleaning
data['sender'] = data['sender'].apply(lambda x: x.lower())
data['receiver'] = data['receiver'].apply(lambda x: x.lower())
data['subject'] = data['subject'].apply(clean_text)
data['body'] = data['body'].apply(clean_text)

stop_words = set(stopwords.words('english'))
additional_stop_words = ['subject', 're', 'edu', 'use', 'http', 'https', 'www', 'html', 'index', 'com', 'net', 'org', 'ect', 'hou', 'cc', 'recipient', 'na', 'pm', 'am', 'et', 'enron']
stop_words = stop_words.union(additional_stop_words)
lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 3 ]
    return ' '.join(filtered_words)

data['body'] = data['body'].apply(tokenize_and_lemmatize)


In [38]:
def remove_html_tags(text):
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

data['body'] = data['body'].apply(remove_html_tags)

In [39]:
def find_number_count(text):
    return len(re.findall(r'\d+', text))

data['number_count'] = data['body'].apply(find_number_count)

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

data['body'] = data['body'].apply(remove_numbers)

In [40]:
data['date'] = pd.to_datetime(data['date'], errors='coerce', utc=True)
if data['date'].dtype == 'datetime64[ns, UTC]':
    data['day_of_week'] = data['date'].dt.day_name()
    data['month'] = data['date'].dt.month
    data['time_of_day'] = data['date'].dt.hour
    data['is_weekend'] = data['date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)
    data['part_of_day'] = data['date'].dt.hour.apply(lambda x: 'morning' if 5 <= x <= 11 else 'afternoon' if 12 <= x <= 17 else 'evening' if 18 <= x <= 22 else 'night')

## Feature Engineering

### Textual Features

In [41]:
def get_mail(text):
    mail = re.findall(r'\<(.*?)\>', text)
    if len(mail) > 0:
        return mail[0]
    else:
        return 0
    
data['sender_mail'] = data['sender'].apply(get_mail)
data['sender'] = data['sender'].apply(lambda x: x.split('<')[0].strip())

In [42]:
# Length of Text
data['body_length'] = data['body'].apply(len)
data['subject_length'] = data['subject'].apply(len)

In [43]:
# Word Count
data['body_word_count'] = data['body'].apply(lambda x: len(x.split()))
data['subject_word_count'] = data['subject'].apply(lambda x: len(x.split()))

In [44]:
# Character Count
data['body_char_count'] = data['body'].apply(lambda x: sum(len(word) for word in x.split()))
data['subject_char_count'] = data['subject'].apply(lambda x: sum(len(word) for word in x.split()))

In [45]:
from textblob import TextBlob

data['body_sentiment'] = data['body'].apply(lambda x: TextBlob(x).sentiment.polarity)
data['subject_sentiment'] = data['subject'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [46]:

suspicious_words = [
    # Avoid spam words that make exaggerated claims and promises
    "100% more", "100% free", "100% satisfied", "Additional income", "Be your own boss",
    "Best price", "Big bucks", "Billion", "Cash bonus", "Cents on the dollar", "Consolidate debt",
    "Double your cash", "Double your income", "Earn extra cash", "Earn money", "Eliminate bad credit",
    "Extra cash", "Extra income", "Expect to earn", "Fast cash", "Financial freedom", "Free access",
    "Free consultation", "Free gift", "Free hosting", "Free info", "Free investment", "Free membership",
    "Free money", "Free preview", "Free quote", "Free trial", "Full refund", "Get out of debt",
    "Get paid", "Giveaway", "Guaranteed", "Increase sales", "Increase traffic", "Incredible deal",
    "Lower rates", "Lowest price", "Make money", "Million dollars", "Miracle", "Money back",
    "Once in a lifetime", "One time", "Pennies a day", "Potential earnings", "Prize", "Promise",
    "Pure profit", "Risk-free", "Satisfaction guaranteed", "Save big money", "Save up to", "Special promotion",

    # Avoid spam words that create unnecessary urgency and pressure
    "Act now", "Apply now", "Become a member", "Call now", "Click below", "Click here",
    "Get it now", "Do it today", "Don’t delete", "Exclusive deal", "Get started now",
    "Important information regarding", "Information you requested", "Instant", "Limited time",
    "New customers only", "Order now", "Please read", "See for yourself", "Sign up free",
    "Take action", "This won’t last", "Urgent", "What are you waiting for?", "While supplies last",
    "Will not believe your eyes", "Winner", "Winning", "You are a winner", "You have been selected",

    # Avoid spam words that look like shady, spammy, or unethical behavior
    "Bulk email", "Buy direct", "Cancel at any time", "Check or money order", "Congratulations",
    "Confidentiality", "Cures", "Dear friend", "Direct email", "Direct marketing", "Hidden charges",
    "Human growth hormone", "Internet marketing", "Lose weight", "Mass email", "Meet singles",
    "Multi-level marketing", "No catch", "No cost", "No credit check", "No fees", "No gimmick",
    "No hidden costs", "No hidden fees", "No interest", "No investment", "No obligation",
    "No purchase necessary", "No questions asked", "No strings attached", "Not junk", "Notspam",
    "Obligation", "Passwords", "Requires initial investment", "Social security number", "This isn’t a scam",
    "This isn’t junk", "This isn’t spam", "Undisclosed", "Unsecured credit", "Unsecured debt",
    "Unsolicited", "Valium", "Viagra", "Vicodin", "We hate spam", "Weight loss", "Xanax",

    # Avoid spam words that are jargon or legalese (and everything else)
    "Accept credit cards", "Ad", "All new", "As seen on", "Bargain", "Beneficiary", "Billing",
    "Bonus", "Cards accepted", "Cash", "Certified", "Cheap", "Claims", "Clearance", "Compare rates",
    "Credit card offers", "Deal", "Debt", "Discount", "Fantastic", "In accordance with laws", "Income",
    "Investment", "Join millions", "Lifetime", "Loans", "Luxury", "Marketing solution", "Message contains",
    "Mortgage rates", "Name brand", "Offer", "Online marketing", "Opt in", "Pre-approved", "Quote",
    "Rates", "Refinance", "Removal", "Reserves the right", "Score", "Search engine", "Sent in compliance",
    "Subject to…", "Terms and conditions", "Trial", "Unlimited", "Warranty", "Web traffic", "Work from home"
]

In [47]:
def count_suspicious_words(text):
    return sum(text.count(word) for word in suspicious_words)

data['suspicious_word_count'] = data['body'].apply(count_suspicious_words)

In [48]:
import pickle

with open('preprocessed_data2.pkl', 'wb') as file:
    pickle.dump(data, file)

print("Data saved successfully!")

Data saved successfully!


In [49]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [50]:
import pickle

preprocessing_pipeline = {
    'clean_text': clean_text,
    'tokenize_and_lemmatize': tokenize_and_lemmatize,
    'remove_html_tags': remove_html_tags,
    'get_mail': get_mail,
    'count_suspicious_words': count_suspicious_words,
    'suspicious_words': suspicious_words,
    'remove_numbers': remove_numbers,
}

with open('preprocessing_pipeline.pkl', 'wb') as file:
    pickle.dump(preprocessing_pipeline, file)

print("Preprocessing pipeline saved successfully!")

Preprocessing pipeline saved successfully!
